## OLP_LLM: Pipeline for generating FOON graphs via LLM Prompting

## Step 1 :- Generate a FOON via LLM Prompting

1. Initialize libraries needed for FOON (``FOON_graph_analyser.py``) as well as OpenAI api.

2. Perform 2-stage prompting for recipe prototype.
    - In the first stage, we ask the LLM for a *high-level recipe* (list of instructions) and a *list of objects* needed for completing the recipe.
    - In the second stage, we ask the LLM for a breakdown of *state changes* that happen for each step of the recipe; specifically, we ask for the *preconditions* and *effects* of each action, which is similar to how a functional unit in FOON has *input* and *output object nodes*.

### Initialize library

In [1]:
%reload_ext autoreload
%autoreload 2

from olp_library import *

### Language to  OLP

In [2]:
models = [
    "gpt-3.5-turbo",
    "gpt-4",
    ]

in_context_file = "incontext_examples.txt"
user_task = "How do I make coffee?"
user_task = "How do I make a Bloody Mary cocktail?"
user_task = "How do I make a Greek salad?"

# user_task = "How do I make a burger?"
# user_task = "How do I unlock a door?"
# user_task = "How do I turn on the lights in my house"

high_level_plan, object_level_plan, plan_objects = generate_olp(user_task, in_context_file, models[1], verbose=True)

print(plan_objects)

*************************************************************************
Stage 1 Prompting
*************************************************************************
Model: gpt-4
Complete prompt:
[
    {
        "role": "system",
        "content": "You are an LLM that understands how to generate concise high level plans for arbitrary tasks involving objects and manipulation. Only focus on what happens to objects. Stay consistent with object names and use one verb per step. Assume all objects needed to complete the plan are available: you do not need to retrieve or clean objects."
    },
    {
        "role": "user",
        "content": "How do I make a Greek salad?\nAfter listing the high level plan, list all the unique objects used in the high level plan, including the object created after solving the task, ignoring state changes to those objects. Follow the format 'unique_objects:object_1, object_2, object_3, ...'"
    }
]
*************************************************************

In [3]:
## Visualize Outputs
print("***********************\n   High level plan\n***********************\n",   high_level_plan,   "\n")
print("***********************\n   Object level plan\n***********************")
print(*object_level_plan, sep="\n")
print("\n***********************\n   Plan objects\n***********************\n",      plan_objects,      "\n")

***********************
   High level plan
***********************
 1. Slice the tomatoes.
2. Slice the cucumbers.
3. Slice the onions.
4. Slice the olives.
5. Crumble the feta cheese.
6. Combine the tomatoes, cucumbers, onions, olives, and feta cheese in a bowl.
7. Drizzle olive oil over the salad.
8. Sprinkle oregano over the salad.
9. Toss the salad to mix the ingredients.

unique_objects: tomatoes, cucumbers, onions, olives, feta cheese, bowl, olive oil, oregano, salad. 

***********************
   Object level plan
***********************
{'Step': 'Slice the tomatoes', 'Objects': ['tomatoes', 'knife'], 'Action': 'Slice', 'StateChanges': {'tomatoes': {'Precondition': ['whole'], 'Effect': ['sliced']}, 'knife': {'Precondition': ['clean'], 'Effect': ['dirty']}}}
{'Step': 'Slice the cucumbers', 'Objects': ['cucumbers', 'knife'], 'Action': 'Slice', 'StateChanges': {'cucumbers': {'Precondition': ['whole'], 'Effect': ['sliced']}, 'knife': {'Precondition': ['dirty'], 'Effect': ['dirty']}}}

In [4]:
## Test Foon creation
plan_step = object_level_plan[0]

sample_unit = create_olp_functionalUnit(plan_step,plan_objects)
sample_unit.print_functions[2](version=1)

Creating functional unit for: Slice the tomatoes
Objects: ['tomatoes', 'knife']
Current object is : tomatoes
	 tomatoes state changes: {'Precondition': ['whole'], 'Effect': ['sliced']}
		 Precondition : whole || related objects: []
		 Effect : sliced || related objects: []
Current object is : knife
	 knife state changes: {'Precondition': ['clean'], 'Effect': ['dirty']}
		 Precondition : clean || related objects: []
		 Effect : dirty || related objects: []
O	tomatoes
S	whole
O	knife
S	clean
M	Slice	<Assumed>
O	tomatoes
S	sliced
O	knife
S	dirty


In [5]:
#Complete FOON prototype:
FOON_prototype = []

for step in object_level_plan:
    print(step['Step'])
    # -- now we will create functional units that follow the FOON format:
    new_unit = create_olp_functionalUnit(step, plan_objects)      
    new_unit.print_functions[2]()
    
    # input()
    print()

    # -- add the functional unit to the FOON prototype:
    FOON_prototype.append(new_unit)

Slice the tomatoes
Creating functional unit for: Slice the tomatoes
Objects: ['tomatoes', 'knife']
Current object is : tomatoes
	 tomatoes state changes: {'Precondition': ['whole'], 'Effect': ['sliced']}
		 Precondition : whole || related objects: []
		 Effect : sliced || related objects: []
Current object is : knife
	 knife state changes: {'Precondition': ['clean'], 'Effect': ['dirty']}
		 Precondition : clean || related objects: []
		 Effect : dirty || related objects: []
O	tomatoes
S	whole
O	knife
S	clean
M	Slice	<Assumed>
O	tomatoes
S	sliced
O	knife
S	dirty

Slice the cucumbers
Creating functional unit for: Slice the cucumbers
Objects: ['cucumbers', 'knife']
Current object is : cucumbers
	 cucumbers state changes: {'Precondition': ['whole'], 'Effect': ['sliced']}
		 Precondition : whole || related objects: []
		 Effect : sliced || related objects: []
Current object is : knife
	 knife state changes: {'Precondition': ['dirty'], 'Effect': ['dirty']}
		 Precondition : dirty || related 

### Writing FOON to a Text File

In [6]:
# -- save the prototype FOON graph as a text file, which we will then run with a parser to correct numbering:
if not os.path.exists('preprocess/'):
	os.makedirs('preprocess/')

if not os.path.exists('postprocess/'):
	os.makedirs('postprocess/')

file_ = open('preprocess/prototype.txt', 'w')
file_.write('#\tFOON Prototype\n#\t-- Task Prompt: {0}\n//\n'.format(user_task))
for unit in FOON_prototype:
	file_.write(unit.getFunctionalUnitText())
	unit.print_functions[2]()
	print('//')

file_.close()

O	tomatoes
S	whole
O	knife
S	clean
M	Slice	<Assumed>
O	tomatoes
S	sliced
O	knife
S	dirty
//
O	cucumbers
S	whole
O	knife
S	dirty
M	Slice	<Assumed>
O	cucumbers
S	sliced
O	knife
S	dirty
//
O	onions
S	whole
O	knife
S	dirty
M	Slice	<Assumed>
O	onions
S	sliced
O	knife
S	dirty
//
O	olives
S	whole
O	knife
S	dirty
M	Slice	<Assumed>
O	olives
S	sliced
O	knife
S	dirty
//
O	feta cheese
S	whole
M	Crumble	<Assumed>
O	feta cheese
S	crumbled
//
O	tomatoes
S	sliced
O	cucumbers
S	sliced
O	onions
S	sliced
O	olives
S	sliced
O	feta cheese
S	crumbled
O	bowl
S	empty
M	Combine	<Assumed>
O	tomatoes
S	in	 [bowl]
O	cucumbers
S	in	 [bowl]
O	onions
S	in	 [bowl]
O	olives
S	in	 [bowl]
O	feta cheese
S	in	 [bowl]
O	bowl
S	contains	{tomatoes,cucumbers,onions,olives,feta cheese}
//
O	olive oil
S	in	 [bottle]
O	salad
S	in	 [bowl]
M	Drizzle	<Assumed>
O	olive oil
S	on	 [salad]
O	salad
S	contains	{olive oil}
//
O	oregano
S	in	 [container]
O	salad
S	contains	{olive oil}
M	Sprinkle	<Assumed>
O	oregano
S	on	 [salad]
O	salad
S	c

## Step 2 :- FOON to PDDL
1. Parse FOON file -- this step is important to ensure that all labels are unique and that the generated file follows the FOON syntax.

2. (Optional) Visualize FOON graph

3. Run ``FOON_to_PDDL.py`` script to generate FOON macro-operators

### Parse and clean generated FOON

In [7]:
# -- running parsing module to ensure that FOON labels and IDs are made consistent for further use:
#		(it is important that each object and state type have a *UNIQUE* identifier)
fpa.skip_JSON_conversion = True		# -- we don't need JSON versions of a FOON
fpa.skip_index_check = True			# -- always create a new set of index files

fpa.source_dir = './preprocess/'	
fpa.target_dir = './postprocess/'
fpa._run_parser()

-- [FOON_parser] : Initiating parsing procedure!


 -- [FOON_parser] : Commencing parsing...
  -- parsing 'prototype.txt'...

 -- [FOON_parser] : Saving corrected files to './postprocess/'...
  -- Saving 'prototype.txt'...

-- Revising object label senses using WordNet and Concept-Net...


SUMMARY OF CHANGES:
  -- new total of OBJECTS : 13
  -- new total of STATES : 10
  -- new total of MOTIONS : 6

 -- [FOON_parser] : Parsing complete!


### (Optional) Run FOON visualization tool

In [8]:
# -- after running the parser, take the 'prototype.txt' file and plot it using the following tool: https://davidpaulius.github.io/foon-view/
# fga._startFOONview()

### Generate PDDL files using ```FOON_to_PDDL``` module

In [11]:
FOON_subgraph_file = './postprocess/prototype.txt'

# -- definition of macro and micro plan file names:
macro_plan_file = os.path.splitext(FOON_subgraph_file)[0] + '_macro.plan'
micro_plan_file = os.path.splitext(FOON_subgraph_file)[0] + '_micro.plan'

# -- create a new folder for the generated problem files and their corresponding plans:
micro_problems_dir = './micro_problems-' + Path(FOON_subgraph_file).stem
if not os.path.exists(micro_problems_dir):
	os.makedirs(micro_problems_dir)

# -- perform conversion of the FOON subgraph file to PDDL:
ftp.FOON_subgraph_file = FOON_subgraph_file
ftp._convert_to_PDDL('OCP')


  -- Reading file line : 100%|██████████| 103/103 [00:00<00:00, 7905.67it/s]


 -- [FOON-fga] : Opening FOON file named './postprocess/prototype.txt'...

 -- [FOON-fga] : Building internal dictionaries...
 -- [FOON-fga] : Building output-to-FU dictionaries...
  -- Level 1: Output object map complete!
  -- Level 2: Output object map complete!
  -- Level 3: Output object map complete!

 -- [FOON-fga] : Building object-to-FU dictionaries...
  -- Level 1: Object map complete!
  -- Level 2: Object map complete!
  -- Level 3: Object map complete!

 -- [FOON_to_PDDL] : Creating domain file named './postprocess/prototype_domain.pddl'...

 -- [FOON_to_PDDL] : Creating problem file named './postprocess/prototype_problem.pddl'...
 -- [FOON_retrieval] : Creating file with kitchen items...


AttributeError: 'NoneType' object has no attribute 'getStatesList'